## Inventors Program Project 2
### Code to combine datasets from each individual well's csv file and the mobilization time csv file


In [1]:
import errno
import pandas as pd
import glob
import csv
import numpy as np 
import os.path
from pathlib import Path

print("Getting ready to work with the dataset")

# get names of all csv files in this folder
csvFiles = glob.glob("./*.csv")

# remove mobilization_time.csv because it has seperate format then 
# all the other data files
csvFiles.remove(".\\mobilization_time.csv")

# if combined.csv exsts from previous run of this script, don't 
# read it as a raw data csv file
if Path("./combined.csv").is_file():
    csvFiles.remove(".\\combined.csv")
    
if Path("./project2_combined.csv").is_file():
    csvFiles.remove(".\\project2_combined.csv")
    
# Print list of csv files
# csvFiles

Getting ready to work with the dataset


In [2]:
print("Creating the map to get company and mobilization details from well name")

# read the mobilization_time file to create a map (dictionary) to get company, and 
# mobilization details given the well_name (to be used later) 
mobcsv = pd.read_csv("./mobilization_time.csv")
mapData = mobcsv.values
mapping = {}

for line in mapData:
    mapping[line[4]] = line[1:4]   

# Print dictionary of well names and [mobilize time (min)	demobilize time (min)	company]
# mapping

Creating the map to get company and mobilization details from well name


In [3]:
# Display headings of individual well csvfiles for reference
heading_list = pd.read_csv(csvFiles[0])
heading_list.head()

,easting,northing,porosity,permeability,Poisson's ratio,Young's Modulus,water saturation,oil saturation,proppant weight (lbs),pump rate (cubic feet/min),time to pump stage (min),set time (min)
0,47700,86600,0.04,0.004,0.313,6947544.661,0.051434,0.948566,1.023477e+06,269.739431,55.6,53
1,47823,86600,0.04,0.006,0.312,6944791.952,0.050877,0.949123,4.037407e+05,352.043256,16.8,55
2,47945,86600,0.04,0.010,0.312,6942377.076,0.050343,0.949657,9.628976e+04,300.275028,4.7,57
3,48068,86600,0.05,0.009,0.312,6938587.300,0.049346,0.950654,8.263443e+05,289.605915,41.8,58
4,48191,86600,0.03,0.003,0.312,6937225.543,0.048885,0.951115,9.734237e+05,311.097261,45.8,59


In [4]:
# Display headings of mobilization time csvfiles for reference
mobcsv.head()

,date,mobilize time (min),demobilize time (min),company,well name
0,2015-01-01,887.0,906.0,1,Brisket 1-204H
1,2015-02-09,949.0,919.0,1,Brisket 1-44H
2,2015-03-09,989.0,886.0,1,Brisket 10-65H
3,2015-03-27,980.0,922.0,1,Brisket 2-149H
4,2015-03-30,878.0,904.0,2,Brisket 3-212H


In [5]:
print("Starting the calculations and combining the dataset...\n")

# open a file to store the combined data, where each row represents a well
with open('project2_combined.csv', 'w') as csvFile:
    
    # write first two colHeads (column header) 
    csvFile.write("Well name,Company")
    
    # write colHeads for well csvfiles
    headings = pd.read_csv(csvFiles[0][2:]).columns
    for colHead in range(2, 12):

        if colHead == 8 or colHead == 10 or colHead == 11:
            # write colHead for parameters which will be summed
            csvFile.write(',Total '+str(headings[colHead]))
            
        else:
            # write colHead for parameters which will be averaged
            csvFile.write(',Average ' + str(headings[colHead]))
    
    # add total fluid and total stages colHeads
    csvFile.write(",Total fluid (cubic feet),Total Stages")
    
    # write colHeads for mobilization time csvfile
    mobHeadings = mobcsv.columns
    csvFile.write(',' + str(mobHeadings[1])) #mobilize time
    csvFile.write(',' + str(mobHeadings[2])) #demobilize time
    
    
    ### Loop through each csv file 
    for f in csvFiles:
        
        # get the well name and company name from the mapping of mobilization csv
        tempData = mapping[f[2:-4]]
        # write first entry of this row (well name) in a new line
        csvFile.write('\n' + f[2:-4])  
        # write second entry of this row (company name)
        # not the first entry, hence write ',' + data
        csvFile.write(',' + str(tempData[2]))   

        # go through individual well csv 
        series = pd.read_csv(f[2:])   # f[2:] is the name of the csv file
        array = series.values         # convert dataframe into numpy array
        
        # loop through each column in well csv
        for col in range(2, 12):
            if col >= 2 and col != 8 and col <= 9:
                # calculate and write average
                csvFile.write(',' + (str(np.average(array[:, col]))))
                
            else:
                # calculate and write sum
                csvFile.write(',' + str(np.sum(array[:, col])))
            
            if col == 11:
                # end of this file
                # calculate and write the total fluid (product and then sum)
                # column 9 is pump rate, column 10 is time to pump
                csvFile.write(',' + str(sum(array[:, 9] * array[:, 10])))
                
                # get and write the number of stages
                csvFile.write(',' + str(array.shape[0]))
                
                # get and write the mobilization and demobilization time from the mapping of mobilization csv
                tempData = mapping[f[2:-4]]
                csvFile.write(',' + str(tempData[0]))
                csvFile.write(',' + str(tempData[1]))
          
                
                # move on to next csv file
                
csvFile.close()

print("Done!")


Starting the calculations and combining the dataset...

Done!


In [6]:
# updating the time col to be normalized

# reopen the combined dataset to calculate the normalized time
combinedData = pd.read_csv("./project2_combined.csv")
combinedDataArray = combinedData.values

# calculate the normalized time and store it in an array
temp = []
for i in range(1, 101):
    temp.append(combinedDataArray[i-1][10] / combinedDataArray[i-1][13] / (combinedDataArray[i-1][12]/10000000) / (combinedDataArray[i-1][8]/10000000))

# add the new calculated array as a column with the following heading
combinedData["Total time to pump stage (min) per stage per 10-million cubic ft fluid per 10-million lb proppant"] = temp

# remove the old time
# combinedData.drop("Total time to pump stage (min)", axis = 1, inplace = True)

# save this updated file with the same name i.e. override the file
combinedData.to_csv("project2_combined.csv", index=False, encoding='utf-8-sig', header = True)

In [7]:
combinedData.head()

,Well name,Company,Average porosity,Average permeability,Average Poisson's ratio,Average Young's Modulus,Average water saturation,Average oil saturation,Total proppant weight (lbs),Average pump rate (cubic feet/min),Total time to pump stage (min),Total set time (min),Total fluid (cubic feet),Total Stages,mobilize time (min),demobilize time (min),Total time to pump stage (min) per stage per 10-million cubic ft fluid per 10-million lb proppant
0,Brisket 1-204H,1,0.038400,0.006040,0.310280,7.097303e+06,0.050455,0.949545,3.934481e+07,295.729046,1963.5,4433.0,576243.872582,50,887.0,906.0,173.207692
1,Brisket 1-44H,1,0.064545,0.017545,0.302636,1.050475e+07,0.101783,0.898217,7.400126e+06,298.734345,447.9,1250.0,130247.654270,11,949.0,919.0,4224.539403
2,Brisket 10-65H,1,0.035000,0.005480,0.325760,6.415031e+06,0.214052,0.785948,4.493244e+07,302.781780,2094.9,4719.0,618604.081893,50,989.0,886.0,150.737222
3,Brisket 2-149H,1,0.054200,0.012720,0.334740,1.202859e+07,0.142279,0.857721,4.005034e+07,298.133691,1904.9,5530.0,556420.737308,50,980.0,922.0,170.959286
4,Brisket 3-212H,2,0.052400,0.011760,0.339640,1.000528e+07,0.091880,0.908120,1.711655e+07,292.153079,834.0,2618.0,242044.473390,25,878.0,904.0,805.219947
